Setup and Preprocessing

In [58]:
# Imports
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import math

# Tensorflow/keras imports
from keras import Sequential
from keras import layers
from keras import regularizers
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight

# Data files
X_train = pd.read_csv("UNSW_NB15_training-set.csv")
X_test = pd.read_csv("UNSW_NB15_testing-set.csv")

# Create train and test dataframes, dropping the id and attack_cat columns from x
# and setting label as y
X_train = X_train.drop(columns=["id", "attack_cat"])
y_train = X_train.pop("label")
X_test = X_test.drop(columns=["id", "attack_cat"])
y_test = X_test.pop("label")

In [59]:
# Function for assigning int values to categorical variables
def handle_non_numerical_data(df: pd.DataFrame) -> pd.DataFrame:
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x += 1

            df[column] = list(map(convert_to_int, df[column]))

    return df

In [60]:
# Use the above function to handle categorical data in the training and testing df
X_train = handle_non_numerical_data(X_train)
X_test = handle_non_numerical_data(X_test)

# Scale the column values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Use class weights to balance the classes for the training set
class_weights = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))

Model structure and training

In [64]:
# Hyperparameters
epochs = 10
batch_size = 32
initial_lr = 0.01

# Function which will decrease the lr by 10% every n epochs
def lr_step_decay(epoch, lr):
    epochs_per_drop = 5
    return initial_lr * math.pow(0.9, math.floor(epoch/epochs_per_drop))

# Model
model = Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(42, activation='relu'),
    layers.Dense(42, activation='relu'),
    layers.Dense(12, activation='relu'),
    layers.Dense(12, activation='relu'),
    layers.Dense(1),
    layers.Activation("sigmoid")
])

model.summary()

# Compile model
model.compile(
    optimizer = SGD(learning_rate=initial_lr, momentum=0.9),
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

# Train model with proper callback
model.fit(X_train, y_train, validation_split=0.2, batch_size=batch_size, epochs=epochs, class_weight=class_weights, verbose=1, callbacks=[LearningRateScheduler(lr_step_decay, verbose=1)])

# Evaluate on test data
print("\nTest data loss/accuracy:")
model.evaluate(X_test, y_test)

4384/4384 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - accuracy: 0.8710 - loss: 0.2587 - val_accuracy: 0.9710 - val_loss: 0.0540 - learning_rate: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/10
4384/4384 ━━━━━━━━━━━━━━━━━━━━ 36s 8ms/step - accuracy: 0.9104 - loss: 0.1678 - val_accuracy: 0.9924 - val_loss: 0.0311 - learning_rate: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.01.
Epoch 3/10
4384/4384 ━━━━━━━━━━━━━━━━━━━━ 36s 8ms/step - accuracy: 0.9146 - loss: 0.1601 - val_accuracy: 0.9855 - val_loss: 0.0440 - learning_rate: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.01.
Epoch 4/10
4384/4384 ━━━━━━━━━━━━━━━━━━━━ 37s 8ms/step - accuracy: 0.9168 - loss: 0.1577 - val_accuracy: 0.9876 - val_loss: 0.0413 - learning_rate: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.01.
Epoch 5/10
4384/4384 ━━━━━━━━━━━━━━━━━━━━ 38s 9ms/step - accuracy: 0.9165 - loss: 0.1580 - val_accuracy: 0.9698 - val_loss: 0.0591 - learni

[0.2555455267429352, 0.8748603463172913]

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_75 (Dense)                │ (None, 42)             │         1,806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 42)             │         1,806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_77 (Dense)                │ (None, 12)             │           516 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_78 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 1)              │            13 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,297 (16.79 KB)

 Trainable params: 4,297 (16.79 KB)

 Non-trainable params: 0 (0.00 B)


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/10
2181/4384 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8490 - loss: 0.3056

KeyboardInterrupt: 